In [1]:
import numpy as np
import pandas as pd
import pickle

from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv('phishing.csv', index_col='Index')
df

,UsingIP,LongURL,ShortURL,Symbol@,Redirecting//,PrefixSuffix-,SubDomains,HTTPS,DomainRegLen,Favicon,...,UsingPopupWindow,IframeRedirection,AgeofDomain,DNSRecording,WebsiteTraffic,PageRank,GoogleIndex,LinksPointingToPage,StatsReport,class
Index,,,,,,,,,,,,,,,,,,,,,
0,1,1,1,1,1,-1,0,1,-1,1,...,1,1,-1,-1,0,-1,1,1,1,-1
1,1,0,1,1,1,-1,-1,-1,-1,1,...,1,1,1,-1,1,-1,1,0,-1,-1
2,1,0,1,1,1,-1,-1,-1,1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
3,1,0,-1,1,1,-1,1,1,-1,1,...,-1,1,-1,-1,0,-1,1,1,1,1
4,-1,0,-1,1,-1,-1,1,1,-1,1,...,1,1,1,1,1,-1,1,-1,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11049,1,-1,1,-1,1,1,1,1,-1,-1,...,-1,-1,1,1,-1,-1,1,1,1,1
11050,-1,1,1,-1,-1,-1,1,-1,-1,-1,...,-1,1,1,1,1,1,1,-1,1,-1
11051,1,-1,1,1,1,-1,1,-1,-1,1,...,1,1,1,1,1,-1,1,0,1,-1


In [3]:
df.columns

Index(['UsingIP', 'LongURL', 'ShortURL', 'Symbol@', 'Redirecting//',
       'PrefixSuffix-', 'SubDomains', 'HTTPS', 'DomainRegLen', 'Favicon',
       'NonStdPort', 'HTTPSDomainURL', 'RequestURL', 'AnchorURL',
       'LinksInScriptTags', 'ServerFormHandler', 'InfoEmail', 'AbnormalURL',
       'WebsiteForwarding', 'StatusBarCust', 'DisableRightClick',
       'UsingPopupWindow', 'IframeRedirection', 'AgeofDomain', 'DNSRecording',
       'WebsiteTraffic', 'PageRank', 'GoogleIndex', 'LinksPointingToPage',
       'StatsReport', 'class'],
      dtype='object')

In [4]:
df.isnull().any()

UsingIP                False
LongURL                False
ShortURL               False
Symbol@                False
Redirecting//          False
PrefixSuffix-          False
SubDomains             False
HTTPS                  False
DomainRegLen           False
Favicon                False
NonStdPort             False
HTTPSDomainURL         False
RequestURL             False
AnchorURL              False
LinksInScriptTags      False
ServerFormHandler      False
InfoEmail              False
AbnormalURL            False
WebsiteForwarding      False
StatusBarCust          False
DisableRightClick      False
UsingPopupWindow       False
IframeRedirection      False
AgeofDomain            False
DNSRecording           False
WebsiteTraffic         False
PageRank               False
GoogleIndex            False
LinksPointingToPage    False
StatsReport            False
class                  False
dtype: bool

In [5]:
df['class'].value_counts()

class
 1    6157
-1    4897
Name: count, dtype: int64

In [6]:
is_phishing = df[df['class'] == 1]
not_phishing = df[df['class'] == -1]

is_phishing_upsampled = resample(is_phishing, n_samples=len(not_phishing))

df_balanced = pd.concat([is_phishing_upsampled, not_phishing])
df_balanced['class'].value_counts()

class
 1    4897
-1    4897
Name: count, dtype: int64

In [7]:
x_data = df_balanced.drop(columns=['class'])
y_data = df_balanced['class']

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_data = scaler.fit_transform(x_data)

In [9]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x_data = scaler.fit_transform(x_data)

In [10]:
linmodel = LinearRegression()

linmodel.fit(x_train, y_train)

with open('lin.pkl', 'wb') as f:
    pickle.dump(linmodel, f)

In [11]:
with open('lin.pkl', 'rb') as f:
    lin_pic = pickle.load(f)

y_pred = lin_pic.predict(x_test)

y_pred_class = np.where(y_pred > 0, 1, -1)

accuracy = accuracy_score(y_test, y_pred_class)
print(f"Accuracy: {accuracy}\n")

report = classification_report(y_test, y_pred_class)
print("Classification Report:")
print(report)

Accuracy: 0.923940786115365

Classification Report:
              precision    recall  f1-score   support

          -1       0.93      0.91      0.92       961
           1       0.92      0.93      0.93       998

    accuracy                           0.92      1959
   macro avg       0.92      0.92      0.92      1959
weighted avg       0.92      0.92      0.92      1959



In [12]:
logmodel = LogisticRegression(solver='newton-cg')

logmodel.fit(x_train, y_train)

with open('log.pkl', 'wb') as f:
    pickle.dump(logmodel, f)

In [13]:
with open('log.pkl', 'rb') as f:
    log_pic = pickle.load(f)

y_pred = log_pic.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}\n")

report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

Accuracy: 0.923940786115365

Classification Report:
              precision    recall  f1-score   support

          -1       0.93      0.92      0.92       961
           1       0.92      0.93      0.93       998

    accuracy                           0.92      1959
   macro avg       0.92      0.92      0.92      1959
weighted avg       0.92      0.92      0.92      1959



In [14]:
param_grid = {
    'n_neighbors': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 20],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'chebyshev'],
}

knn = KNeighborsClassifier()

grid_search = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy')

grid_search.fit(x_train, y_train)

params = grid_search.best_params_

print("Best Hyperparameters:", params)

KeyboardInterrupt: 

In [ ]:
knn = KNeighborsClassifier(n_neighbors=params['n_neighbors'], weights=params['weights'], metric=params['metric'])

knn.fit(x_train, y_train)

with open('knn.pkl', 'wb') as f:
    knn_pic = pickle.dump(knn, f)

In [ ]:
with open('knn.pkl', 'rb') as f:
    knn_pic = pickle.load(f)

y_pred = knn_pic.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier




estimators = [
    ('knn', KNeighborsClassifier()),
    ('rf', RandomForestClassifier(random_state=42)),
    ('gb', GradientBoostingClassifier(random_state=42))
]

stacking_model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())


stacking_model.fit(x_train, y_train)

y_pred = stacking_model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Stacking Accuracy: {accuracy}")
print(f"Classification Report\n{report}")


In [ ]:

from sklearn.model_selection import train_test_split


x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.10, random_state=42)


knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy with adjusted test size: {accuracy}")
print(f"Classification Report\n{report}")